In [1]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

%matplotlib inline

In [6]:
# Load .env environment variables
load_dotenv("/Users/FinTech/PycharmProjects/Bootcamp/api_keys.env")

True

In [7]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
type(alpaca_api_key)

str

In [8]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [10]:
# Format current date as ISO format
end_date = pd.Timestamp("2021-04-30", tz="America/New_York").isoformat()
start_date = pd.Timestamp("2018-04-30", tz="America/New_York").isoformat()

In [87]:
# Set the tickers
tickers = ["SPY", "LQD", "UNG", "PFE", "VNQ"]

In [88]:
# Set timeframe to one day ('1D') for the Alpaca API
timeframe = "1D"

In [89]:
etfs = alpaca.get_barset(
    tickers,
    timeframe,
    start = start_date,
    end = end_date,
    limit = 1000
).df

# Display sample data
etfs.isna().sum()


LQD  open      0
     high      0
     low       0
     close     0
     volume    0
PFE  open      0
     high      0
     low       0
     close     0
     volume    0
SPY  open      0
     high      0
     low       0
     close     0
     volume    0
UNG  open      0
     high      0
     low       0
     close     0
     volume    0
VNQ  open      0
     high      0
     low       0
     close     0
     volume    0
dtype: int64